<a href="https://colab.research.google.com/github/patelami3431/Web-Crawler/blob/master/HLT_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import urllib
from urllib import request
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import re

In [66]:
url = "https://www.medicalnewstoday.com/articles/5847.php"
p = request.urlopen(url).read()
soup = BeautifulSoup(p, 'html.parser')


# row = soup.find(class_ = 'c-entry-content')
counter = 0
s = []
for link in soup.find_all('a'):
        s.append(link.get('href'))
        counter += 1
        
print(counter)

182


In [60]:
counter = 0
final = []
for link in s:
    if link and ('medical' in link or 'diet' in link) and ('https' in link or 'http' in link):
        print(link)
        final.append(link)
        counter += 1
        
print(counter)

https://www.medicalnewstoday.com/privacy-policy#vendors
https://www.medicalnewstoday.com/privacy-policy
https://www.medicalnewstoday.com/advertising
https://www.medicalnewstoday.com/privacy-policy
http://www.medicalnewstoday.com/articles/7379.php
http://www.medicalnewstoday.com/articles/7382.php
http://www.medicalnewstoday.com/articles/180858.php
http://www.medicalnewstoday.com/articles/8749.php
http://www.medicalnewstoday.com/articles/149636.php
http://www.medicalnewstoday.com/articles/149454.php
http://www.medicalnewstoday.com/articles/7380.php
http://www.medicalnewstoday.com/articles/7381.php
http://www.medicalnewstoday.com/articles/149090.php
http://www.nhs.uk/Livewell/Goodfood/Pages/what-is-a-Mediterranean-diet.aspx
https://www.medicalnewstoday.com/articles/5847.php
https://www.pinterest.com/search/pins/?q=www.medicalnewstoday.com
https://www.medicalnewstoday.com/articles/5847.php
https://www.medicalnewstoday.com
18


In [70]:
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element.encode('utf-8'))):
        return False
    return True

In [73]:
url2 = "http://www.medicalnewstoday.com/articles/149636.php"
p2 = request.urlopen(url).read()
soup2 = BeautifulSoup(p2, 'html.parser')
data = soup2.find_all(text = True)
result = filter(visible, data)
temp_list = list(result) 
temp_str = ' '.join(temp_list)
temp_str
# page = soup.find('p').getText()

'\n [5847|cccs|]   \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n \n [if lt IE 9]>\n<script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>\n\n\t\t\t<link rel="stylesheet" href="https://cdn-prod.medicalnewstoday.com/structure/styles/out/mnt_ie.css" type="text/css" media="all" />\n\t\n\n\n\t<![endif] \n [if IE 7]>\n\n<script src="//ie7-js.googlecode.com/svn/version/2.1(beta4)/IE7.js"></script>\n<![endif] \n [if IE 9]>\n<script src="//ie7-js.googlecode.com/svn/version/2.1(beta4)/IE9.js"></script>\n<![endif] \n desktop \n \n \n \n \n \n <script type=\'text/javascript\'>\n(function() {\nvar useSSL = \'https:\' == document.location.protocol;\nvar src = (useSSL ? \'https:\' : \'http:\') +\n\'//www.googletagservices.com/tag/js/gpt.js\';\ndocument.write(\'<scr\' + \'ipt src="\' + src + \'"></scr\' + \'ipt>\');\n})();\n</script> \n \n  <script async src="//js-sec.indexww.com/ht/p/184901-208206248649346.js"></script>  